In [61]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

In [62]:
df_USA = pd.read_csv('USA_20-22.csv')
print(df_USA.head())

         DATE  JHDUSRGDPBR
0  2020-01-01          1.0
1  2020-04-01          1.0
2  2020-07-01          0.0
3  2020-10-01          0.0
4  2021-01-01          0.0


In [63]:
df_UK = pd.read_csv('UK_monthly_gdp.csv')
print(df_UK.head())

  Time Period  GDP Growth
0    /01/2020         0.3
1    /02/2020        -0.5
2    /03/2020        -7.0
3    /04/2020       -20.9
4    /05/2020         3.2


# Montly Data GDP_USA for 1947- 2022

In [64]:
fig = go.Figure(data=go.Heatmap(
                   z=[df_USA['JHDUSRGDPBR']],
                   x=df_USA.index,
                   y=['GDP'],
                   colorscale='Viridis'))

fig.update_layout(title='GDP Growth over Time',
                  xaxis_title='Time Period',
                  yaxis_title='JHDUSRGDPBR')

fig.show()

# Montly Data GDP_UK for 2020-2022

In [79]:
fig = go.Figure(data=go.Heatmap(
                   z=[df_UK['GDP Growth']],
                   x=df_USA.index,
                   y=['GDP Growth'],
                   colorscale='inferno'))

fig.update_layout(title='GDP Growth over Time',
                  xaxis_title='Time Period',
                  yaxis_title='')

fig.show()

In [66]:
#UK
# Convert monthly data to quarterly data using resample method
df_UK['Time Period'] = pd.to_datetime(df_UK['Time Period'], format='/%m/%Y')
df_UK.set_index('Time Period', inplace=True)
quarterly_data_UK = df_UK.resample('Q').mean()
print(quarterly_data_UK.head())

             GDP Growth
Time Period            
2020-03-31    -2.400000
2020-06-30    -2.900000
2020-09-30     3.500000
2020-12-31     0.200000
2021-03-31     0.033333


In [67]:
quarterly_data_UK

,GDP Growth
Time Period,
2020-03-31,-2.400000
2020-06-30,-2.900000
2020-09-30,3.500000
2020-12-31,0.200000
2021-03-31,0.033333
2021-06-30,1.800000
2021-09-30,0.433333
2021-12-31,0.400000
2022-03-31,0.066667


In [70]:
quarterly_data_UK

,GDP Growth
Time Period,
2020-03-31,-2.400000
2020-06-30,-2.900000
2020-09-30,3.500000
2020-12-31,0.200000
2021-03-31,0.033333
2021-06-30,1.800000
2021-09-30,0.433333
2021-12-31,0.400000
2022-03-31,0.066667


In [80]:
#UK
# Calculate recession based on quarterly GDP growth
quarterly_data_UK['Recession'] = ((quarterly_data_UK['GDP Growth'] < 0) & (quarterly_data_UK['GDP Growth'].shift(1) < 0))

# Fill missing values with False (since the first quarter cannot be in a recession)
quarterly_data_UK['Recession'].fillna(False, inplace=True)

# Plot the GDP growth and recession data
fig = go.Figure()
fig.add_trace(go.Scatter(x=quarterly_data_UK.index, 
                         y=quarterly_data_UK['GDP Growth'], 
                         name='GDP Growth', 
                         line=dict(color='Blue', width=2)))
fig.add_trace(go.Scatter(x=quarterly_data_UK[quarterly_data_UK['Recession']].index, 
                         y=quarterly_data_UK[quarterly_data_UK['Recession']]['GDP Growth'], 
                         name='Recession', line=dict(color='red', width=2)))

fig.update_layout(title='GDP Growth and Recession over Time (Quarterly Data)',
                  xaxis_title='Time Period',
                  yaxis_title='GDP Growth')

fig.show()

In [91]:
from matplotlib import colormaps
quarterly_data_UK['Recession Start'] = quarterly_data_UK['Recession'].ne(quarterly_data_UK['Recession'].shift()).cumsum()
recession_periods = quarterly_data_UK.groupby('Recession Start')
recession_duration = recession_periods.size()
recession_severity = recession_periods['GDP Growth'].sum()

fig = go.Figure()
fig.add_trace(go.Bar(x=recession_duration.index, y=recession_duration,
                     name='Recession Duration'))
fig.add_trace(go.Bar(x=recession_severity.index, y=recession_severity,
                     name='Recession Severity'))

fig.update_layout(title='Duration and Severity of Recession',
                  xaxis_title='Recession Periods',
                  yaxis_title='Duration/Severity')

fig.show()